# Contextually Propagated Term Weights for Document Representation

## Load reuters training/test data

In [21]:
# load training data
train_file = open("data/reuters/r8-train-all-terms.txt", "r")
train_data = []
train_labels = []
for line in train_file:
    splitted = line.split()
    train_labels.append(splitted[0])
    words = " ".join(splitted[1:])
    train_data.append(words)

In [22]:
#loading test data
test_file = open("data/reuters/r8-test-all-terms.txt", "r")
test_data = []
test_labels = []
for line in test_file:
    splitted = line.split()
    test_labels.append(splitted[0])
    words = " ".join(splitted[1:])
    test_data.append(words)

## TF-IDF baseline

In [20]:
from sklearn.neighbors import NearestNeighbors 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 

x_train = train_texts

# CountVectorizer for transforming the text 
cv = CountVectorizer()
X_train_counts = cv.fit_transform(training) 
tfidf = TfidfTransformer() 
train_tfidf = tfidf.fit_transform(X_train_counts)

# apply knn for evaluation purposes
knn = NearestNeighbors(n_neighbors=1, n_jobs=-1) 
knn.fit(train_tfidf)

test= test_texts 
X_test_counts = count_vect.transform(test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

comp = knn.kneighbors(X_test_tfidf, return_distance=False)

### TF-IDF Scores

In [9]:
from sklearn.metrics import f1_score
pred_labels = [train_labels[int(index)] for index in comp]
print("micro:", f1_score(test_labels, pred_labels, average = "micro"))
print("macro:", f1_score(test_labels, pred_labels, average = "macro"))

micro: 0.840109639104614
macro: 0.7862565745939916


## CPTW

In [ ]:
def gamma(w_k, d_i, w_j):
    freq = d_i.split().count(w_k)
    sim = model.similarity(w_k, w_j)
    return freq * sim

def cptw(d_i, embedding):
    for w in d_i.split()